In [1]:
# 네이버_모두를 위한 딥러닝
# 텐서플로를 통한 자동차 연비 예측하기 (mpg 파일)

### 데이터셋 로드

In [2]:
import pandas as pd
import seaborn as sns

In [3]:
df = sns.load_dataset('mpg')
df.shape

(398, 9)

In [4]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


### 결측치 확인

In [5]:
df.isna()  # 결측치 확인
df.isna().sum()  # 결측치 개수 확인

# 결측치 개수가 많지 않으므로 제거를 고려

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
name            0
dtype: int64

### 결측치 제거

In [8]:
df = df.dropna()
df.shape

# 6개 (결측)데이터 제거된 것을 확인

(392, 9)

### 수치 데이터만 가져오기
* 머신러닝이나 딥러닝은 모델 내부에서 수치계산을 하므로 숫자가 아닌 데이터는 학습과 예측을 할 수 없다

In [10]:
# select_dtypes를 통해 object 타입을 제거하고 가져옴
df = df.select_dtypes(exclude = 'object')
df.shape

# 2개의 열 제거

(392, 7)

### 전체 데이터 기술 통계 확인

In [11]:
df.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


### 데이터셋 나누기

In [12]:
# 전체 df에서 train, test를 분리
train_dataset = df.sample(frac = 0.8, random_state=42)  

# sample을 사용해서 분리 가능
# random_state로 랜덤 분리값 고정

train_dataset.shape

(314, 7)

In [13]:
test_dataset = df.drop(train_dataset.index)  # index
test_dataset.shape

(78, 7)

In [14]:
# train_dataset, test_dataset에서 label(정답) 값을 꺼내 label을 따로 생성
# 문제에서 정답을 분리하는 과정

train_labels = train_dataset.pop('mpg')
test_labels = test_dataset.pop('mpg')

print(train_labels.shape, test_labels.shape)

(314,) (78,)


In [ ]:
train_dataset.head(2)

# mpg 열 제거 확인

In [ ]:
test_labels.head(2)

---

## 딥러닝 모델 만들기

In [ ]:
# tensorflow 불러오기
import tensorflow as tf
tf.__version__

### 딥러닝 층 구성

In [ ]:
model = tf.keras.Sequential()
# input_shape에 변수의 개수 입력
model.add(tf.keras.layers.Dense(64, input_shape=[len(train_dataset.keys())]))
model.add(tf.keras.layers.Dense(64)) # 히든레이어 1 생성
model.add(tf.keras.layers.Dense(64)) # 히든레이어 2 생성
model.add(tf.keras.layers.Dense(1))          

### 모델 컴파일


In [ ]:
# loss : 회귀모델 평가에 mse 사용
# metrics : 모델의 정확도 판단, 회귀의 경우 mae, mse, rmse등을 사용
model.compile(loss='mse', metrics=['mae', 'mse'])

### 만들어진 모델 확인

In [ ]:
model.summary()

# 2개의 히든레이어가 있는 model임을 확인 할 수 있다

---

### 딥러닝 모델로 학습

In [ ]:
model.fit(train_dataset, train_labels, epochs=10, verbose=1)

### 딥러닝 모델로 평가

In [ ]:
model.evaluate(test_dataset, test_labels)

### 딥러닝 모델의 예측

In [ ]:
predict_labels = model.predict(test_dataset).flatten()  # flatten : 다차원 배열을 1차원으로 평평하게 해줌
predict_labels

### 딥러닝 모델의 예측결과 평가

In [ ]:
sns.scatterplot(x=test_labels, y=predict_labels)

In [ ]:
# 학습(.fit)할 때 epochs를 높이는 방법
# 딥러닝 층(Dense)에 activation = 'relu' 옵션 추가